**Heidelberg University**

**Data Science  Group**
    
Prof. Dr. Michael Gertz  

Ashish Chouhan, Satya Almasian, John Ziegler, Jayson Salazar, Nicolas Reuter
    
January 16, 2024
    
Natural Language Processing with Transformers

Winter Semster 2023/2024     
***

# **Assignment 4: Question Answering**
**Due**: Monday, January 29, 2024, 2pm, via [Moodle](https://moodle.uni-heidelberg.de/course/view.php?id=19251)



### **Submission Guidelines**

- Solutions need to be uploaded as a **single** Jupyter notebook. You will find several pre-filled code segments in the notebook, your task is to fill in the missing cells.
- For the written solution, use LaTeX in markdown inside the same notebook. Do **not** hand in a separate file for it.
- Download the .zip file containing the dataset but do **not** upload it with your solution.
- It is sufficient if one person per group uploads the solution to Moodle, but make sure that the full names of all team members are given in the notebook.

***

## **Task 1: Retrieval Augmented Generation (RAG)** ( 4.5 + 3 + 4 + 3 + 1.5 = 16 points)

In this task, we look at using the open source `Llama-13b-chat` model for creating a RAG system. You must first apply for access to Llama 2 models via [this](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) form (access is typically granted within a few hours). etrieval augmented generation you also need to request to use the model on Hugging Face by going to the [model](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) card. ***Note that the emails you provide for your Hugging Face account must match the email you used to request Llama 2.***

The final piece that you need is a Hugging Face authentication token. You can find such a token by going to the `setting` in your Hugging Face profile, under the `Access Token` menu you can generate a new token.

To store the document you will need a free Pinecone [API key](https://app.pinecone.io/).
Make sure you have these pieces ready before starting to work on this task.

----
When ready, let's start by downloading the necessary packages.

It is advised to proceed with this notebook with a GPU (if you are on Colab make sure that a GPU environment is activated.)


Place all the access tokens in the `.env` file and upload it to the working directory (if you are running this notebook locally, you can change the path to fit your working directory). Please use the following format:


```
HF_AUTH= "Hugging Face Authentication Key"
PINECONE_API_KEY="Pincone API Key"
PINECONE_ENVIRONMENT="Pinecone Environment"
```

Run the cell below to load the access tokens into the environment variables.

In [1]:
# Stop the kernel
#os.kill(os.getpid(), 9)

In [5]:
%pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [7]:
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
%pip install xformers
%pip install bitsandbytes
%pip install matplotlib seaborn tqdm
%pip install chromadb
%pip install evaluate
%pip install rouge_score
%pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.23.post1 requires torch==2.1.2, but you have torch 2.1.0 which is incompatible.
  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7



## Subtask 1.1: Data Preparation



We need a collection of documents to perform our retrieval on. To make it closer to your final project, you will be downloading and using a subset of the LangChain documentation. We get some of the `.html` files located on the site. The code below will download all HTML files from the links on the webpage into a `docs` directory. `-l1` limits the download to only the first level of depth.


In [15]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!wget -r -l1 -A.html -P docs https://api.python.langchain.com/en/stable/langchain_api_reference.html

 The docs are going to be used as input text for answering questions that a normal language model might not be aware of (LangChain docs is not necessarily part of its training data of Llama2). We can use LangChain itself to process these docs. Use the [ReadTheDocsLoader](https://python.langchain.com/docs/integrations/document_loaders/readthedocs_documentation) to load the docs from the `docs` folder.

 At the time of creating this notebook, there  `423` documents were downloaded. However, since the documentation is being updated regularly this number might be different for you.

In [16]:
from langchain.document_loaders import ReadTheDocsLoader

# Load LangChain docs
docs = ReadTheDocsLoader("docs").load()
len(docs)

418

Let's take a look at one of the documents. You see that LangChain has created a `Document` object. Look at the example below and fill in the cells to print out the text content and URL of the page (the URL of the page should starts with `https://`).

In [17]:
docs[10]

Document(page_content='langchain_anthropic 0.0.1.post2¶\nlangchain_anthropic.chat_models¶\nClasses¶\nchat_models.ChatAnthropicMessages\nBeta ChatAnthropicMessages chat model.\nFunctions¶', metadata={'source': 'docs/api.python.langchain.com/en/stable/anthropic_api_reference.html'})

In [18]:
# Explore a document
doc_10 = docs[10]
page_content = doc_10.page_content
page_url = "https://"+doc_10.metadata.get('source', '')[5:]  # Access source URL from metadata

# Print page content and URL
print("page content:\n", page_content)
print("\npage url:\n", page_url)


page content:
 langchain_anthropic 0.0.1.post2¶
langchain_anthropic.chat_models¶
Classes¶
chat_models.ChatAnthropicMessages
Beta ChatAnthropicMessages chat model.
Functions¶

page url:
 https://api.python.langchain.com/en/stable/anthropic_api_reference.html


As you can imagine the documents can be long and if multiple of them are required as context to answer questions, we need to take the document lengths into account.
This is due to the fact that language models do not have unlimited context span. In our case, we plan to use Llama2 for this project, where the maximum token limit is 4096. This limit is not only the input but also takes the generated output into account, moreover, you need to leave room for the query and instructions as well. Therefore, it is important to chunk the longer documents into smaller-sized fragments.

Based on your use case and how many contexts you plan to feed into the model the length of these fragments will differ.
In this case, we choose to assign 2000 tokens to context and choose to generate the answer from 5 context fragments, which leaves us with 400 tokens per context fragment as the maximum chunk size.

To count the number of tokens in a chunk, we need to load the correct tokenizer for Llama2. Fill the code cell below to load the correct tokenizer and use it to complete the function that counts the number of tokens per given chunk.

**Hint:** you need to use your Hugging Face authentication token to load the tokenizer.

In [3]:
#If you get an error here during the first import from the `transformers` package, restart the kernel and try again.
from transformers import AutoTokenizer

# Get the Hugging Face token from the environment
hugging_face_token = os.environ.get('HF_AUTH')
model_name = 'meta-llama/Llama-2-13b-chat-hf'

# Load the correct tokenizer for Llama2 using your Hugging Face token
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hugging_face_token)


In [19]:
# Count the number of tokens in a chunk
def token_len(text):
    return len(tokenizer.encode(text, truncation=True, max_length=4096))


Count the number of tokens for all documents and use it to compute minimum, maximum, and average token count statistics across all documents. Depending on how the documentation is updated by the time you run the cell below the numbers might slightly differ.

In [20]:
# Count tokens for all documents
token_counts = [token_len(doc.page_content) for doc in docs]

# Compute statistics
min_tokens = min(token_counts)
avg_tokens = sum(token_counts) / len(token_counts)
max_tokens = max(token_counts)

print(f"""Min: {min_tokens}
Avg: {avg_tokens}
Max: {max_tokens}""")

Min: 49
Avg: 1891.1459330143541
Max: 4096


Now we will use LangChain's built-in chunking functionality to split the text into smaller chunks. LangChain offers a variety of text splitters that you can check out [here](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.text_splitter).
Use the general-purpose splitter that splits text by recursively looking at characters. Use this class to split the text into 400 token-sized chunks, where the length of each chunk is computed based on the `token_len` function. The length is not the only criterion for splitting, if any of these separators `'\n\n', '\n', ' ', ''` is encountered, we will have a new chunk.
Since splitting only based on maximum length might result in incoherent chunks for every consecutive chunk, let the chunk overlap by 50 tokens. This way,  we preserve some of the previous context while chunking.

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm

# Load recursive-character text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=['\n\n', '\n', ' ', ''], chunk_size=400, chunk_overlap=50)



In [22]:
chunks = text_splitter.split_text(docs[100].page_content)
len(chunks)

132

In [23]:
token_len(chunks[0])

128

The next step is to apply the splitting function to all the documents in our corpus and to save our chunks in a logical way. We also want to assign a unique ID to each chunk so we know which part of the documentation they come from. In the end, the corpus should be transformed into a list of dictionaries of the following format:


```
[
    {
        "id": "glossary-0",
        "text": "first chunk of the document glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    },
    {
        "id": "glossary-1",
        "text": "second chunk of glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    }
    ...
]
```

Construct the IDs by taking the name of the page before the suffix `.html` and appending a chronological number indicating which chunk it is.


In [24]:
from tqdm.auto import tqdm

documents = []

for doc in tqdm(docs):
    url = "https://"+doc.metadata.get('source', '')[5:]
    uid = url.split("/")[-1].split(".html")[0]  # Extract page name before ".html"
    chunks = text_splitter.split_text(doc.page_content)

    # Construct data in the required format
    for i, chunk in enumerate(chunks):
        chunk_id = f"{uid}-{i}"
        documents.append({
            "id": chunk_id,
            "text": chunk,
            "source": url
        })

len(documents) # once again this value might differ based on how the LangChain documentation is updated


  0%|          | 0/418 [00:00<?, ?it/s]

19245

For the next steps, we require a `DataFrame`.

In [25]:
import pandas as pd
data = pd.DataFrame(documents)
data.head()

,id,text,source
0,mistralai_api_reference-0,langchain_mistralai 0.0.3¶\nlangchain_mistrala...,https://api.python.langchain.com/en/stable/mis...
1,google_genai_api_reference-0,langchain_google_genai 0.0.6.post1¶\nlangchain...,https://api.python.langchain.com/en/stable/goo...
2,google_genai_api_reference-1,Google Generative AI Embeddings.\nlangchain_go...,https://api.python.langchain.com/en/stable/goo...
3,core_api_reference-0,langchain_core 0.1.16¶\nlangchain_core.agents¶...,https://api.python.langchain.com/en/stable/cor...
4,core_api_reference-1,Functions¶\nlangchain_core.beta¶\nClasses¶\nbe...,https://api.python.langchain.com/en/stable/cor...


#### ${\color{red}{Comments\ 1.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.2: Document Embedding Pipeline


In this task, we initialize the embedding pipeline to transform the chunks into vector embeddings using Hugging Face and LangChain. These embeddings are used for similarity search between the query and the chunks to retrieve the most relevant chunks.
  We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding, which is a rather small model that you can easily run on Colab. Initialize the model using `HuggingFaceEmbeddings` to use Hugging Face via Langchain. The encoding batch size should be 32, and make sure that the model is placed on the correct device, otherwise, this can take a long time.

In [26]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
import pinecone
from tqdm import tqdm

In [27]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0' # make sure you are on gpu

docs = [
    "An example document",
    "A second document as an example"
]


# Set model and encoding parameters
model_kwargs = {'device': 'cuda:0'}  # Specify the CUDA device
encode_kwargs = {'normalize_embeddings': False, 'batch_size': 32}

# Initialize the model
embed_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)


Embed the example documents using the model you created and check the output.
The output should be a list of lists, containing the embeddings.

In [28]:
#Embed the example documents
embeddings = embed_model.embed_documents(docs)

print("number of docs:",len(embeddings))
print("dimension of docs:", len(embeddings[0]))

number of docs: 2
dimension of docs: 384


Now we use the embedding pipeline created above to store the embeddings in a Pinecone vector index. First, lets setup the Pinecone environment, collect your API key and environment name from the environment variables, and initiate Pinecone with them.

In [29]:
import os
from pinecone import Pinecone, ServerlessSpec

# Get Pinecone API key and environment from environment variables
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Create an instance of the Pinecone class
pc = Pinecone(api_key=pinecone_api_key)


Initialize the index `rag-assignment` inside Pinecone. Use the cosine similarity as similarity metric. Keep in mind that if you run this multiple times on a free tier, where only one index is allowed, you need to remove the index created to make room for a new one (Pinecone index gets archived automatically after 14 days of inactivity).

In [62]:
index_name = 'rag-assignment'
if index_name not in pc.list_indexes().names():
  pc.create_index(
    name=index_name,
    dimension=len(embeddings),
    metric='cosine',
    spec=ServerlessSpec(
      cloud="aws",
      region="us-central1"
    )
  )

Lets take a look at the index you created. As of now the index should be empty but have the correct embedding dimension.

In [63]:
index_name = 'rag-assignment'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Process the dataset in batches of `32` and push the vectors to the Pinecone index. Your index should include the IDs and embeddings for each chunk. As metadata, pass the original text as `text` and the URL as `source` (no need to add the `https`). We use this metadata later to retrieve the original text.

In [64]:
# Push vectors to the Pinecone index
batch_size = 32

for i in tqdm(range(0, len(data), batch_size)):
    vectors = []

    # Extract data for the current batch
    batch_data = data.iloc[i:i+batch_size]

    # Process each row in the batch
    for _, row in batch_data.iterrows():
        vector = {}

        # Get information from the current row
        ids = row['id']
        texts = row['text']
        source_url = row['source']

        # Embed the text using your embedding model
        embeds = embed_model.embed_query(texts)

        # Create the vector with id, values (embeddings), and metadata
        vector["id"] = ids
        vector["values"] = embeds
        vector["metadata"] = {"text": texts, "source": source_url}

        # Append the vector to the list
        vectors.append(vector)

    # Upsert the vectors to the Pinecone index
    index.upsert(vectors=vectors, namespace="ns1")


100%|██████████| 602/602 [04:08<00:00,  2.42it/s]


Now if we look at the index statistics we should have vectors of dimension `384`.

In [48]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.19094,
 'namespaces': {'ns1': {'vector_count': 19094}},
 'total_vector_count': 19094}

#### ${\color{red}{Comments\ 1.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.3: Text Generation Pipeline


So far we have our index ready and a way to find the most similar chunks to our query. Now, we need a way to generate the answer from the retrieved chunks. For this purpose, we use the `text-generation` pipeline from Hugging Face (refer to the Hugging Face [tutorial](https://moodle.uni-heidelberg.de/pluginfile.php/1286642/mod_resource/content/1/HuggingFace.ipynb)) and load it into LangChain using a wrapper.

In [4]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'meta-llama/Llama-2-13b-chat-hf'

Quantization techniques reduce memory and computational costs by representing weights and activations with lower-precision data types like 8-bit integers (int8). This enables loading larger models you normally wouldn’t be able to fit into memory, and thus speeds up inference.
To make the process of model quantization more accessible, Hugging Face has seamlessly integrated with the [Bitsandbytes](https://huggingface.co/docs/accelerate/usage_guides/quantization) library.

Define a config from `Bitsandbytes` that enables 4-bit quantization and set the nested quantization to `true`. This changes the datatype from float 32 (default) to normalized float 4 datatype to contain 4 bits of information.
Additionally, add a compute type to store weights in 4-bits, but the computation to happen in 16-bit (bfloat16).
Moreover, set the `bnb_4bit_use_double_quant` to true, which uses a second quantization after the first one to save an additional 0.4 bits per parameter.
Refer to [here](https://huggingface.co/docs/transformers/main_classes/quantization) for more information.

In [5]:
  ### your code ###
from transformers import BitsAndBytesConfig

bitsAndBites_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype = 'bfloat16',
    bnb_4bit_quant_type = 'fp4',
    bnb_4bit_use_double_quant = True
)
  ### your code ###

Use your Hugging Face token to load the correct model configuration using the `transformers` library.

In [6]:
### your code ###
from transformers import AutoConfig

access_token = os.environ.get('HF_AUTH')

model_config = AutoConfig.from_pretrained(model_id, token=access_token)
### your code ###


Load the model for text generation (pay attention to the model type) using the configuration file you have defined, with the specified quantization, and set the `trust_remote_code` flag to `true`. Another flag that is useful for large mode is  `device_map="auto"`. By setting this flag, Accelerate will determine where to put each layer to maximize the use of GPUs and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM).

It will take a while for the model to download.

In [7]:
#Loading the model will take some time, (roughly 5 min)
### your code ###
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=access_token,
    config=model_config,
    quantization_config=bitsAndBites_config,
    trust_remote_code=True,
    device_map="auto"
)
### your code ###
model.eval()# we only use the model for inference
print(f"Model loaded ")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Model loaded 


You can even check the memory footprint of your model using the `get_memory_footprint` method.


In [8]:
model.get_memory_footprint()

7083970560

The next thing we need to do is initialize a `text-generation` pipeline with Hugging Face that uses the Llama2 model to generate some text, given some input. We will then use this pipeline inside LangChain to build our question-answering system.
`text-generation` pipeline generates text from a language model conditioned on a given input. The pipeline is similar to other Hugging Face pipelines and requires two things that we must initialize:

1.   A language model, in this case, it will be `meta-llama/Llama-2-13b-chat-hf`.
2.   A tokenizer for the language model.

LangChain expects the full-text outputs, therefore set the `return_full_text` to true. You can also pass additional generation parameters to the model.
Since we want the questions to be answered mainly based on the retrieved chunks, let's set the model temperature to a low value of 0.01 to reduce randomness. Additionally, add a repetition penalty of 1.1 to stop the model from repeating itself and the maximum number of generation tokens to 512.

In [9]:
### your code ###
from transformers import pipeline

generate_text = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    temperature=0.01,
    repetition_penalty=1.1,
    max_new_tokens=512
)
### your code ###

We provide the language model a general question to make sure our pipeline is working correctly.

In [10]:
sample_input="Explain to me the difference between alligator and crocodile."
### your code ###

generated_text = generate_text(sample_input)
### your code ###
print(generated_text)

[{'generated_text': 'Explain to me the difference between alligator and crocodile.\n\nAlligators and crocodiles are both large, carnivorous reptiles that live in aquatic environments, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape, while crocodiles are more streamlined.\n2. Habitat: Alligators are found only in freshwater environments, such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Nesting habits: Alligators build mounds of vegetation and mud to lay their eggs, while crocodiles dig holes in the sand or mud to lay the

Use the LangChain Hugging Face wrapper, as subset of [LLM chain](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) to create an interface for the text generation pipeline.

In [11]:
### your code ###
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)
### your code ###

To confirm that it works the same way, use the sample input to generate text using the llm chain. The input should be passed as the `prompt` to the language model.

In [13]:
### your code ###
response = llm(sample_input)
response
### your code ###

'\n\nAlligators and crocodiles are both large, carnivorous reptiles that live in aquatic environments, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape, while crocodiles are more streamlined.\n2. Habitat: Alligators are found only in freshwater environments, such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Nesting habits: Alligators build mounds of vegetation and mud to lay their eggs, while crocodiles dig holes in the sand or mud to lay their eggs.\n5. Jaw structure: Alligators have a different jaw structure than crocod

#### ${\color{red}{Comments\ 1.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.4: Question Answering Chain


For Retrieval Augmented Generation (RAG) in LangChain, we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object.

`RetrievalQA` is a method for question-answering tasks, utilizing an index to retrieve relevant documents or text chunks, it is suitable for straightforward Q&A applications.

`RetrievalQAWithSourcesChain` is an extension of RetrievalQA that chains together multiple sources of information, providing context and the source for answers.

 For both of these, we need an LLM and a Pinecone index. For LangChain to be able to use the Pinecone index, we need to initialize it through the LangChain vector store.

 **Hint**: You need to explicitly tell the vector storage where to find the original text.

In [49]:
from langchain.vectorstores import Pinecone
### your code ###
vectorstore = Pinecone.from_existing_index(index_name=index_name, embedding=embed_model, namespace="ns1")
### your code ###

Let's try a query that is specific to the LangChain documentation and see which chunks are relevant. Use the vector storage defined above to find the top-3 chunks related to the given query.

In [55]:
query = 'what is a LangChain Agent?'
### your code ###
vectorstore.similarity_search(query, 3)
### your code ###

[Document(page_content='langchain.agents.agent.Agent¶\nclass langchain.agents.agent.Agent[source]¶\nBases: BaseSingleActionAgent\n[Deprecated]  Agent that calls the language model and deciding the action.\nThis is driven by an LLMChain. The prompt in the LLMChain MUST include\na variable called “agent_scratchpad” where the agent can put its', metadata={'source': 'https://api.python.langchain.com/en/stable/agents/langchain.agents.agent.Agent.html'}),
 Document(page_content='langchain 0.1.4¶\nlangchain.agents¶\nAgent is a class that uses an LLM to choose a sequence of actions to take.\nIn Chains, a sequence of actions is hardcoded. In Agents,\na language model is used as a reasoning engine to determine which actions\nto take and in which order.\nAgents select and use Tools and Toolkits for actions.\nClass hierarchy:\nBaseSingleActionAgent --> LLMSingleActionAgent', metadata={'source': 'https://api.python.langchain.com/en/latest/langchain_api_reference.html'}),
 Document(page_content='lan

Now use the `vectorstore` and `llm` to initialize the `RetrievalQA` object, which showcases question answering over an index. `RetrievalQA` is a document chain, these are useful for summarizing documents, answering questions about documents, extracting information from documents, and more. All such chains operate with 4 different chain types:


1.   `stuff`: it takes a list of documents, inserts them all into a prompt, and passes that prompt to an LLM.
2.   `refine`: it constructs a response by looping over the input documents and iteratively updating its answer. It is well-suited for tasks that require analyzing more documents than can fit in the model’s context.
3. `map_reduce`:  it first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combined documents chain to get a single output (the Reduce step).
4. `map_re_rank`: it runs an initial prompt on each document that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest-scoring response is returned.

For this assignment, we focus only on the first type. Make sure to set the `verbose` to `true`, so we can see the different stages of processing that happens while answering a question (you might need to set this parameter more than once). As mentioned before, we want our retrieve to input top-5 most similiar chunks to the query to generate an answer.

In [61]:
from langchain.chains import RetrievalQA
### your code ###

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}, verbose=True),
    verbose=True,
    return_source_documents=True
)

### your code ###
query='what is a LangChain Agent?'

First, we try to answer the question only using Llama2. As you see the answer is not convincing as it does not have access to the LangChain documentation.

In [54]:
llm(query)

'\n\nA LangChain Agent is an AI-powered chatbot that uses natural language processing (NLP) to understand and respond to user queries. It is designed to provide personalized support and answer questions in real-time, 24/7. The agent is trained on a large dataset of customer interactions, which enables it to understand the nuances of human language and provide accurate responses.\n\nLangChain Agents are powered by advanced machine learning algorithms that allow them to learn from each interaction and improve their performance over time. They can be integrated with various messaging platforms such as Facebook Messenger, WhatsApp, Slack, and more. This allows businesses to provide seamless support to their customers across multiple channels.\n\nSome of the key features of LangChain Agents include:\n\n1. Natural Language Processing (NLP): LangChain Agents use NLP to understand and interpret user queries, allowing them to provide accurate responses.\n2. Machine Learning: LangChain Agents ar

Now use the Pipeline from above and see how the answer changes.

In [64]:
### your code ###
result = rag_pipeline({"query": query})
result["result"]
### your code ###




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

langchain.agents.agent.Agent¶
class langchain.agents.agent.Agent[source]¶
Bases: BaseSingleActionAgent
[Deprecated]  Agent that calls the language model and deciding the action.
This is driven by an LLMChain. The prompt in the LLMChain MUST include
a variable called “agent_scratchpad” where the agent can put its

langchain 0.1.4¶
langchain.agents¶
Agent is a class that uses an LLM to choose a sequence of actions to take.
In Chains, a sequence of actions is hardcoded. In Agents,
a language model is used as a reasoning engine to determine which actions
to take and in which order.
Agents select and use Tools and Toolkits for actions.
Class hierarchy:
BaseSingleActionAgent --> LLMSingleA

' A LangChain Agent is a type of agent in the LangChain framework that uses a language model to reason about the sequence of actions it should take. It is driven by an LLMChain, which includes a variable called "agent_scratchpad" where the agent can put its own state. The agent selects and uses tools and toolkits for actions. There are different types of LangChain Agents, such as Conversational Agents, which hold conversations in addition to using tools.\n\nPlease let me know if my understanding is correct or not.'

#### ${\color{red}{Comments\ 1.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.5: Conversational Retrieval Chain




We can also extend our retrieval chain to be able to remember the previous questions and answer the current question by looking at the previous context.
The important part of a conversational model is conversation memory, which transforms the stateless language model to be able to remember previous interactions, e.g., similiar to ChatGPT. In this subtask, we will use LangChain to create a conversational memory.


To implement the memory we use `ConversationalRetrievalChain`.
This chain takes in chat history (a list of messages) and new questions and then returns an answer to that question. The algorithm for this chain consists of three parts:

1. Use the chat history and the new question to create a new question that contains the information from the previous context.

2. This new question is passed to the retriever and relevant documents are returned.

3. The retrieved documents are passed to an LLM to generate a final response.

In [73]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
chat_history = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### your code ###
qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}, verbose=True),
    memory=chat_history,
    verbose=True
)
result = qa_conversation({"question": query})
### your code ###
result["answer"]




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

langchain.agents.agent.Agent¶
class langchain.agents.agent.Agent[source]¶
Bases: BaseSingleActionAgent
[Deprecated]  Agent that calls the language model and deciding the action.
This is driven by an LLMChain. The prompt in the LLMChain MUST include
a variable called “agent_scratchpad” where the agent can put its

langchain 0.1.4¶
langchain.agents¶
Agent is a class that uses an LLM to choose a sequence of actions to take.
In Chains, a sequence of actions is hardcoded. In Agents,
a language model is used as a reasoning engine to determine which actions
to take and in which order.
Agents select and use Tools and Toolkits for actions.
Class hierarchy:
BaseSingleActionAge

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


' A LangChain Agent is a type of agent in the LangChain framework that uses a language model to reason about the sequence of actions it should take. It is driven by an LLMChain, which includes a variable called "agent_scratchpad" where the agent can put its own state. The agent selects and uses tools and toolkits for actions. There are different types of LangChain Agents, such as Conversational Agents, which hold conversations in addition to using tools.\n\nPlease let me know if my understanding is correct or not.'

Change the chat history to contain the previous question and answer pair and ask a follow-up question.  

In [76]:
follow_up="What are tools and toolkits?"

### your code ###
result = qa_conversation({"question": follow_up})
### your code ###



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: what is a LangChain Agent?
Assistant:  A LangChain Agent is a type of agent in the LangChain framework that uses a language model to reason about the sequence of actions it should take. It is driven by an LLMChain, which includes a variable called "agent_scratchpad" where the agent can put its own state. The agent selects and uses tools and toolkits for actions. There are different types of LangChain Agents, such as Conversational Agents, which hold conversations in addition to using tools.

Please let me know if my understanding is correct or not.
Follow Up Input: What are tools and toolkits?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain cha

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


This is the previous context that was fed in alongside the new question.

In [77]:
chat_history

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what is a LangChain Agent?'), AIMessage(content=' A LangChain Agent is a type of agent in the LangChain framework that uses a language model to reason about the sequence of actions it should take. It is driven by an LLMChain, which includes a variable called "agent_scratchpad" where the agent can put its own state. The agent selects and uses tools and toolkits for actions. There are different types of LangChain Agents, such as Conversational Agents, which hold conversations in addition to using tools.\n\nPlease let me know if my understanding is correct or not.'), HumanMessage(content='What are tools and toolkits?'), AIMessage(content='  In the context of LangChain Agents, tools and toolkits refer to pre-built functionality that can be used by agents to perform specific tasks or actions. These tools and toolkits can be loaded into the agent using the `load_tools` function, and can be accessed throug

The current question is answered by knowing that the tools and toolkits are referring to a LangChain Agent, which was part of the previous question.

In [78]:
result['answer']

'  In the context of LangChain Agents, tools and toolkits refer to pre-built functionality that can be used by agents to perform specific tasks or actions. These tools and toolkits can be loaded into the agent using the `load_tools` function, and can be accessed through the `get_all_tool_names` function to determine what tools are available. Agents can then use these tools to perform actions such as sending messages or making requests to external services.'

#### ${\color{red}{Comments\ 1.5}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 2: Advanced RAG Techniques and Evaluation (4 + 5 = 9 points)**

Now that you have successfully implemented your first RAG system, we dive into more advanced techniques and learn how to evaluate your methods using metrics you learned during the lecture. We focus on evaluation with an already annotated dataset. To this end, we load a small subset of [NarrativeQA](https://huggingface.co/datasets/narrativeqa), which is an English-language dataset of stories and corresponding questions designed to test reading comprehension, especially on long documents. We only load 30 samples from the data, as you will see in the upcoming sections, answer generation takes quite some time. In actual setting, it is advised to use a much larger set to obtain statistically significant results.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("satyaalmasian/narrativeqa_subset",split="train[:30]")
len(dataset)

Since we already used our free index in Pinecone for the previous task, we use Chroma, an open-source vector database, instead. As opposed to Pinecone, Chroma creates a collection on your machine.

In [ ]:
from langchain.docstore.document import Document
documents=[ doc["text"] for doc in dataset["document"]]
questions=[quest for quest in dataset["question"]]
answers=[ans for ans in dataset["answers"]]
documents=list(set(documents))

In [ ]:
docs= [Document(page_content=doc, metadata={"source": "local"}) for doc in documents]

The number of documents is smaller  than the number of questions and answers and each document is used as a reference for multiple questions:

In [ ]:
print(len(docs))
print(len(questions))

##Subtask 2.1: Build Contextual Compression in LangChain

Let's split our documents using the TextSplitter from Task 1 and embed them inside the Chroma database with the embedding model of the previous task.

In [ ]:
### your code ###
all_splits =
### your code ###

In [ ]:
from langchain.vectorstores import Chroma
### your code ###
vectordb =
retriever =
### your code ###

In [ ]:
print("Fist question in the set:",questions[2]['text'])
r_docs = retriever.get_relevant_documents(questions[2]['text'])
r_docs

First, make a simple RAG pipeline that works on top of the Chroma retriever. This retriever should be similar to the previous task. However, since we want to use it for a large number of questions, remove the `verbose` parameters.

In [ ]:
from langchain.chains import RetrievalQA
### your code ###
rag_simple=
### your code ###

We look at an example question and compare the answer by RAG to the gold answer from the dataset. Note that the answers can contain multiple lines.

In [ ]:
rag_simple(questions[2]['text']) #ignore the warning

In [ ]:
answers[2]

Apply the `rag_simple` pipeline to all the question in your corpus and accumulate the answers. **It should take around 10 minutes on a T4 GPU on Colab**.

In [ ]:
simple_answers=[]
### your code ###

### your code ###

Libraries such as LangChain and [Llamaindex](https://www.llamaindex.ai/) provide a variety of retrieval strategies for building a RAG system. In this subtask, you will use one of these variations called **contextual compression**. This method aims to extract only the relevant information from documents, reducing the need for expensive language model calls and improving response quality. Contextual compression consists of two parts:


1.  **Base retriever:** retrieves the initial set of documents based on the query. This is similar to the retriever from the previous task.
2.   **Document compressor:** processes these documents to extract the relevant content. We use `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter
from langchain.llms import OpenAI

### your code ###

compression_retriever =
### your code ###

Let's take a look at an example of compression retriever works.

In [ ]:
print("Fist question in the set:",questions[2]['text'])
compressed_docs = compression_retriever.get_relevant_documents(questions[2]['text'])
compressed_docs

Look at the output and try out several different questions by yourself. Does the compressed output make sense?

Compare this to the previous **simple** approach. Which one, in your opinion, is better?

Finally, we use the new retriever with the Llama2 model from the previous task to create the context compressor RAG pipeline. The code below should be similiar to what you did in the previous task. Once again, make sure to turn off the `verbose` argument.

In [ ]:
### your code ###
from langchain.chains import RetrievalQA

rag_compressor =
### your code ###


In [ ]:
rag_compressor(questions[2]['text'])

Now we can use the pipeline to generate answers for all the questions in our dataset. **It should take around 20 minutes on a T4 GPU on Colab.**

In [ ]:
compressor_answers=[]
### your code ###

### your code ###


#### ${\color{red}{Comments\ 2.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

##Subtask 2.2. Evaluate

Since we have access to ground truth answers, we can use various evaluation metrics from the literature. In this task, we explore three metrics:


1.   **BLEU:** BLEU score stands for Bilingual Evaluation Understudy and is a precision-based metric developed
for evaluating machine translation. BLEU scores a candidate by computing the
number of n-grams in the candidate that also appear
in a reference. The n can vary, in this task we compute for n=4.
2.   **ROUGE:** ROUGE score stands for Recall-Oriented Understudy for Gisting Evaluation and is an F-measure metric designed for
evaluating translation and summarization. There are a number of variants of ROUGE.
3. **BERTScore:** BERTScore first obtains BERT representation of each word in the candidate and reference by feeding the candidate
and reference through a BERT model separately.
An alignment is then computed between candidate
and reference words by computing pairwise cosine
similarity. This alignment is then aggregated in to
precision and recall scores before being aggregated
into a (modified) F1 score that is weighted using
inverse-document-frequency values.

Luckily, Hugging Face has an implementation for all these metrics. Use the `evaluate` library to load the metrics.

Use the loaded metrics to compare the RAG pipelines from the previous subtask.

In [ ]:
import evaluate
### your code ###
bleu =
rouge =
bertscore =
### your code ###

As seen in the previous subtask, the answers can contain multiple lines. To be able to compare the output of our systems to the gold answers, merge the multiple answers into a single string.

In [ ]:
answers_merged=[]
### your code ###

### your code ###
print(len(answers_merged))

Compute the BLUE score for the simple RAG and compressor RAG.

In [ ]:
### your code ###
bleu_simple =
bleu_compressor =
### your code ###
print("Simple system:")
print(bleu_simple)
print("Compressor:")
print(bleu_compressor)

What does the elements below in the output of the BLEU impelementation in Hugging Face mean? (do not copy and paste the documentation but write the implications behind each element!).



1.   **precisions:** `your answer`
2.   **brevity_penalty:** `your answer`
3.   **translation_length:** `your answer`
4.   **reference_length:** `your answer`
5.   **length_ratio:** `your answer`




**Answer:**


1.   **precisions:** precision of n-grams, which is calculated as the number of n-grams that appear in both the machine-generated translation and the reference translations divided by the total number of n-grams in the machine-generated translation.
2.   **brevity_penalty:** is a penalty term that adjusts the score for translations that are shorter than the reference translations. It is calculated as min(1, (reference_length / translation_length)). It essentially penalizes generated translations that are too short compared to the closest reference length with an exponential decay.
3.   **translation_length:**   is the total number of words in the machine-generated translation.
4.   **reference_length:**  is the total number of words in the reference translations.
5. **length_ratio:** ratio of the 3 and 4.

In [ ]:
### your code ###
rouge_simple =
rouge_compressor =
### your code ###
print("Simple system:")
print(rouge_simple)
print("Compressor:")
print(rouge_compressor)

What is the difference in variants of ROUGE (ROUGE-N, ROUGE-L, ROUGE-SUM)?

`your answer`


**Answer:**

ROUGE measures the similarity between the machine-generated summary and the reference summaries using overlapping n-grams, word sequences that appear in both the machine-generated summary and the reference summaries. The most common n-grams used are unigrams, bigrams, and trigrams. ROUGE score calculates the recall of n-grams in the machine-generated summary by comparing them to the reference summaries.

**ROUGE-N:** ROUGE-N measures the overlap of n-grams (contiguous sequences of n words) between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the n-gram overlap. For example, ROUGE-1 (unigram) measures the overlap of single words, ROUGE-2 (bigram) measures the overlap of two-word sequences, and so on. ROUGE-N is often used to evaluate the grammatical correctness and fluency of generated text.

**ROUGE-L:** ROUGE-L measures the longest common subsequence (LCS) between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the length of the LCS. ROUGE-L is often used to evaluate the semantic similarity and content coverage of generated text, as it considers the common subsequence regardless of word order.

**ROUGE-S:** ROUGE-S measures the skip-bigram (bi-gram with at most one intervening word) overlap between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the skip-bigram overlap. ROUGE-S is often used to evaluate the coherence and local cohesion of generated text, as it captures the semantic similarity between adjacent words.



In [ ]:
import numpy as np
bertscore_simple_averaged={}
bertscore_compressor_averaged={}
### your code ###


### your code ###
print("Simple system:")
print(bertscore_simple_averaged)
print("Compressor:")
print(bertscore_compressor_averaged)

Which model works better?

#### ${\color{red}{Comments\ 2.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$